<a href="https://colab.research.google.com/github/kaviya68/kaviya68/blob/main/HAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub


path = kagglehub.dataset_download("die9origephit/human-activity-recognition")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/die9origephit/human-activity-recognition/versions/6


In [ ]:
import os

data_path = "/root/.cache/kagglehub/datasets/die9origephit/human-activity-recognition/versions/6"
print(os.listdir(data_path))


['time_series_data_human_activities.csv', 'zzzread.txt']


In [ ]:
import pandas as pd

data_path = "/root/.cache/kagglehub/datasets/die9origephit/human-activity-recognition/versions/6/time_series_data_human_activities.csv"

df = pd.read_csv(data_path)
print(df.shape)
print(df.head())


(1073623, 6)
   user activity      timestamp  x-axis  y-axis  z-axis
0     1  Walking  4991922345000    0.69   10.80   -2.03
1     1  Walking  4991972333000    6.85    7.44   -0.50
2     1  Walking  4992022351000    0.93    5.63   -0.50
3     1  Walking  4992072339000   -2.11    5.01   -0.69
4     1  Walking  4992122358000   -4.59    4.29   -1.95


In [ ]:
import pandas as pd

data_path = "/root/.cache/kagglehub/datasets/die9origephit/human-activity-recognition/versions/6/time_series_data_human_activities.csv"
df = pd.read_csv(data_path)


df = df.dropna()


df = df[['activity', 'x-axis', 'y-axis', 'z-axis']]
print(df.head())


  activity  x-axis  y-axis  z-axis
0  Walking    0.69   10.80   -2.03
1  Walking    6.85    7.44   -0.50
2  Walking    0.93    5.63   -0.50
3  Walking   -2.11    5.01   -0.69
4  Walking   -4.59    4.29   -1.95


In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler


le = LabelEncoder()
df['activity'] = le.fit_transform(df['activity'])

scaler = StandardScaler()
scaled_features = scaler.fit_transform(df[['x-axis', 'y-axis', 'z-axis']])


In [ ]:
import numpy as np

def create_sequences(X, y, time_steps=50):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

X, y = create_sequences(scaled_features, df['activity'].values)
print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (1073573, 50, 3)
y shape: (1073573,)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential([
    LSTM(64, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(len(np.unique(y)), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        17,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,686 (76.90 KB)

 Trainable params: 19,686 (76.90 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/10
13420/13420 ━━━━━━━━━━━━━━━━━━━━ 458s 34ms/step - accuracy: 0.8723 - loss: 0.3656 - val_accuracy: 0.9768 - val_loss: 0.0792
Epoch 2/10
13420/13420 ━━━━━━━━━━━━━━━━━━━━ 495s 33ms/step - accuracy: 0.9788 - loss: 0.0727 - val_accuracy: 0.9858 - val_loss: 0.0476
Epoch 3/10
13420/13420 ━━━━━━━━━━━━━━━━━━━━ 454s 34ms/step - accuracy: 0.9862 - loss: 0.0482 - val_accuracy: 0.9905 - val_loss: 0.0330
Epoch 4/10
13420/13420 ━━━━━━━━━━━━━━━━━━━━ 490s 33ms/step - accuracy: 0.9895 - loss: 0.0371 - val_accuracy: 0.9923 - val_loss: 0.0273
Epoch 5/10
13420/13420 ━━━━━━━━━━━━━━━━━━━━ 407s 30ms/step - accuracy: 0.9913 - loss: 0.0308 - val_accuracy: 0.9928 - val_loss: 0.0246
Epoch 6/10
13420/13420 ━━━━━━━━━━━━━━━━━━━━ 448s 33ms/step - accuracy: 0.9926 - loss: 0.0260 - val_accuracy: 0.9946 - val_loss: 0.0184
Epoch 7/10
13420/13420 ━━━━━━━━━━━━━━━━━━━━ 420s 31ms/step - accuracy: 0.9936 - loss: 0.0228 - val_accuracy: 0.9941 - val_loss: 0.0197
Epoch 8/10
13420/13420 ━━━━━━━━━━━━━━━━━━━━ 434s 32ms/s

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")


6710/6710 ━━━━━━━━━━━━━━━━━━━━ 62s 9ms/step - accuracy: 0.9962 - loss: 0.0139
Test Accuracy: 0.9961
